In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
# Load the CSV file with results from the Python survey into a data frame.
py_filename = '../data/2020_sharing_data_outside.csv'

py_df = pd.read_csv(py_filename, low_memory=False)

In [4]:
# rename some columns

general_columns = ['age',
                   'are.you.datascientist',
                   'is.python.main',
                   'company.size',
                   'country.live',
                   'employment.status',
                   'first.learn.about.main.ide',
                   'how.often.use.main.ide',
                   'is.python.main',
                   'main.purposes'
                   'missing.features.main.ide'
                   'nps.main.ide',
                   'python.version.most',
                   'python.years',
                   'python2.version.most',
                   'python3.version.most',
                   'several.projects',
                   'team.size',
                   'use.python.most',
                   'years.of.coding'
                  ]

# Use the function `pd.MultiIndex.from_tuples` to create the multi-index, 
# and then reassign it back to `df.columns`. 

def column_multi_name(column_name):
    if column_name in general_columns:
        return ('general', column_name)
    else:
        first, rest = column_name.rsplit('.', 1)
        return (first, rest)
    
py_df.columns = pd.MultiIndex.from_tuples([column_multi_name(one_column_name)
                  for one_column_name in py_df.columns    ])

In [5]:
# Sort the columns, such that they're in alphabetical order. 

py_df = py_df[sorted(py_df.columns)]

In [6]:
py_df[('ide', 'main')].value_counts().head(10)

(ide, main)
VS Code                         8010
PyCharm Professional Edition    5144
PyCharm Community Edition       3815
Vim                             2176
Sublime Text                    1201
Jupyter Notebook                1167
Atom                             784
Other                            711
Emacs                            636
Spyder                           580
Name: count, dtype: int64

In [7]:
# Which 10 other programming languages are most commonly used by Python developers?

(
    py_df['other.lang']
    .count()
    .sort_values(ascending=False)
    .head(10)
)

JavaScript      16662
HTML/CSS        15469
Bash / Shell    13793
SQL             13391
C/C++           11623
Java             8109
C#               4460
PHP              4060
TypeScript       3717
Other            3592
dtype: int64

In [8]:
# What were the 10 most common countries from which survey participants came?
(
    py_df[('general', 'country.live')]
    .value_counts()
    .head(10)
)

(general, country.live)
United States         3975
India                 2800
Germany               1807
China                 1155
United Kingdom        1110
France                1078
Russian Federation     935
Other country          880
Brazil                 812
Canada                 644
Name: count, dtype: int64

In [9]:
(
    py_df[('general', 'python.years')]
    .value_counts(normalize=True)
)

(general, python.years)
3–5 years           0.284272
Less than 1 year    0.239542
1–2 years           0.224834
6–10 years          0.154939
11+ years           0.096413
Name: proportion, dtype: float64

In [10]:
# Which country has the greatest number of Python developers with 11+ years of experience?
(
    py_df['general']
    [py_df[('general','python.years')] == '11+ years']
    .groupby('country.live')['python.years']
    .count()
    .sort_values(ascending=False)
    .head(1)
)

country.live
United States    691
Name: python.years, dtype: int64

In [11]:
# Proportion of Python devs per country with 11+years experience

country_experience = py_df['general'][['country.live', 'python.years']]
all_per_country = country_experience['country.live'].value_counts() 

In [12]:
expert_per_country = (
    country_experience.loc[
        country_experience['python.years'] == '11+ years',
        'country.live'
    ]
    .value_counts()
)

country.live
United States         3975
India                 2800
Germany               1807
China                 1155
United Kingdom        1110
                      ... 
Kazakhstan              36
Dominican Republic      34
Uruguay                 33
Armenia                 31
Uzbekistan              31
Name: count, Length: 76, dtype: int64

In [13]:
(expert_per_country / all_per_country).sort_values(ascending=False).dropna().head(10)


country.live
Norway            0.265432
Ireland           0.225490
Australia         0.225420
Belgium           0.225108
Slovenia          0.224490
New Zealand       0.197917
Sweden            0.194030
Finland           0.190141
United Kingdom    0.186486
Austria           0.186170
Name: count, dtype: float64

In [14]:
# Load the CSV file with results from the Stack Overflow survey into a data frame.
so_filename = '../data/so_2021_survey_results.csv'

so_df = pd.read_csv(so_filename, low_memory=False)

In [16]:
so_df.head(3)

ResponseId                                         MainBranch  \
0           1                     I am a developer by profession   
1           2             I am a student who is learning to code   
2           3  I am not primarily a developer, but I write co...   

                                          Employment             Country  \
0  Independent contractor, freelancer, or self-em...            Slovakia   
1                                 Student, full-time         Netherlands   
2                                 Student, full-time  Russian Federation   

  US_State UK_Country                                            EdLevel  \
0      NaN        NaN  Secondary school (e.g. American high school, G...   
1      NaN        NaN       Bachelor’s degree (B.A., B.S., B.Eng., etc.)   
2      NaN        NaN       Bachelor’s degree (B.A., B.S., B.Eng., etc.)   

      Age1stCode                                          LearnCode YearsCode  \
0  18 - 24 years  Coding Bootcamp;Other online resources (ex: vi...       NaN   
1  11 - 17 years  Other online resources (ex: videos, blogs, etc...         7   
2  11 - 17 years  Other online resources (ex: videos, blogs, etc...       NaN   

   ...              Age Gender Trans                Sexuality  \
0  ...  25-34 years old    Man    No  Straight / Heterosexual   
1  ...  18-24 years old    Man    No  Straight / Heterosexual   
2  ...  18-24 years old    Man    No        Prefer not to say   

                      Ethnicity      Accessibility       MentalHealth  \
0  White or of European descent  None of the above  None of the above   
1  White or of European descent  None of the above  None of the above   
2             Prefer not to say  None of the above  None of the above   

            SurveyLength SurveyEase ConvertedCompYearly  
0  Appropriate in length       Easy             62268.0  
1  Appropriate in length       Easy                 NaN  
2  Appropriate in length       Easy                 NaN  

[3 rows x 48 columns]

In [17]:
# Show the average salary for different types of employment. 
# Contractors and freelancers like to say that they earn more than full-time employees. 
# What does the data here show us?

(
    so_df
    .groupby('Employment')['ConvertedCompYearly'].mean()
    .sort_values(ascending=False)
    .dropna()
    .apply(lambda n: f'{n:,.2f}')
)

Employment
I prefer not to say                                     1,455,643.25
Employed full-time                                        121,369.67
Independent contractor, freelancer, or self-employed      107,433.97
Retired                                                    69,533.25
Employed part-time                                         41,136.12
Name: ConvertedCompYearly, dtype: object

In [18]:
# Create a pivot table in which the index contains countries, 
# the columns are education levels, 
# and the cells contain the average salary for each education level per country.

pd.options.display.float_format = '{:,.2f}'.format
(
    so_df # 原始数据
    .pivot_table(index='Country',   # 行索引：按国家分组
                 columns='EdLevel',  # 列索引：按教育水平分组
                 values='ConvertedCompYearly') # 值：转换后的年薪
)

EdLevel                               Associate degree (A.A., A.S., etc.)  \
Country                                                                     
Afghanistan                                                           NaN   
Albania                                                               NaN   
Algeria                                                               NaN   
Andorra                                                               NaN   
Angola                                                                NaN   
...                                                                   ...   
Venezuela, Bolivarian Republic of...                                  NaN   
Viet Nam                                                         7,827.00   
Yemen                                                                 NaN   
Zambia                                                                NaN   
Zimbabwe                                                        36,000.00   

EdLevel                               Bachelor’s degree (B.A., B.S., B.Eng., etc.)  \
Country                                                                              
Afghanistan                                                              30,288.00   
Albania                                                                  19,152.86   
Algeria                                                                  21,770.67   
Andorra                                                                  94,045.50   
Angola                                                                   31,500.00   
...                                                                            ...   
Venezuela, Bolivarian Republic of...                                     30,108.77   
Viet Nam                                                                 18,463.11   
Yemen                                                                     5,628.67   
Zambia                                                                   40,173.00   
Zimbabwe                                                                  8,399.27   

EdLevel                               Master’s degree (M.A., M.S., M.Eng., MBA, etc.)  \
Country                                                                                 
Afghanistan                                                             10,176,704.00   
Albania                                                                     80,127.62   
Algeria                                                                     15,052.57   
Andorra                                                                     22,056.00   
Angola                                                                            NaN   
...                                                                               ...   
Venezuela, Bolivarian Republic of...                                        28,680.00   
Viet Nam                                                                    50,599.80   
Yemen                                                                             NaN   
Zambia                                                                       4,908.00   
Zimbabwe                                                                          NaN   

EdLevel                               Other doctoral degree (Ph.D., Ed.D., etc.)  \
Country                                                                            
Afghanistan                                                                  NaN   
Albania                                                                      NaN   
Algeria                                                                      NaN   
Andorra                                                               146,981.00   
Angola                                                                       NaN   
...                                                                          ...   
Venezuela, Bolivarian Republic of...             

用 pandas 的 pivot_table 创建一个透视表，并设置数字的显示格式为 带千位分隔符的两位小数（如 1,234.56

- 设置数字显示格式: `pd.options.display.float_format = '{:,.2f}'.format`

In [19]:

# Load the CSV file with OECD data
oecd_filename = '../data/oecd_locations.csv'

oecd_df = pd.read_csv(oecd_filename, header=None, index_col=1, names=['abbrev', 'Country'])

In [21]:
# Create this pivot table again, only including countries in our OECD subset.
(
    oecd_df
    .join(so_df.set_index('Country'))
    .pivot_table(index='Country',
                 columns='EdLevel', 
                 values='ConvertedCompYearly')
)

EdLevel    Associate degree (A.A., A.S., etc.)  \
Country                                          
Australia                           117,049.64   
Austria                              43,623.38   
Belgium                              35,664.00   
Brazil                               25,347.42   
Canada                               87,930.35   
Denmark                              80,217.33   
Finland                             282,353.67   
France                               54,394.89   
Germany                              98,530.52   
Hungary                              51,041.00   
Israel                              146,420.90   
Italy                                36,427.94   
Japan                               143,196.83   

EdLevel    Bachelor’s degree (B.A., B.S., B.Eng., etc.)  \
Country                                                   
Australia                                    180,794.16   
Austria                                       66,096.13   
Belgium                                       68,474.70   
Brazil                                        47,681.23   
Canada                                       140,668.24   
Denmark                                       80,430.94   
Finland                                       69,381.34   
France                                        65,062.38   
Germany                                      102,751.23   
Hungary                                       48,756.45   
Israel                                       124,973.40   
Italy                                         73,697.22   
Japan                                         85,821.47   

EdLevel    Master’s degree (M.A., M.S., M.Eng., MBA, etc.)  \
Country                                                      
Australia                                       106,794.32   
Austria                                          77,645.66   
Belgium                                          88,580.41   
Brazil                                           42,056.01   
Canada                                          144,733.35   
Denmark                                         115,751.85   
Finland                                          74,360.98   
France                                           94,585.73   
Germany                                         110,611.98   
Hungary                                          46,771.71   
Israel                                          128,492.71   
Italy                                            79,132.19   
Japan                                           130,143.19   

EdLevel    Other doctoral degree (Ph.D., Ed.D., etc.)  \
Country                                                 
Australia                                  150,234.96   
Austria                                     74,783.17   
Belgium                                     80,832.44   
Brazil                                      43,123.21   
Canada                                     102,989.35   
Denmark                                    102,785.19   
Finland                                     61,508.25   
France                                     140,402.86   
Germany                                    108,718.46   
Hungary                                     52,833.60   
Israel                                     131,812.62   
Italy                                       93,490.78   
Japan                                      157,239.40   

EdLevel    Primary/elementary school  Professional degree (JD, MD, etc.)  \
Country                                                                    
Australia                 153,327.50                          108,725.00   
Austria                    86,877.00                           38,915.33   
Belgium                11,342,349.25                           71,000.20   
Brazil                      7,880.00                           25,449.67   
Canada                     73,787.83                           82,953.40   
Denmark                    91,056.57        

In [22]:
# In which of these countries does someone with an associate degree earn the most? 
(
    oecd_df
    .join(so_df
          .set_index('Country'))
    .pivot_table(index='Country',
                 columns='EdLevel', 
                 values='ConvertedCompYearly')['Associate degree (A.A., A.S., etc.)']
    .sort_values(ascending=False)
)

Country
Finland     282,353.67
Israel      146,420.90
Japan       143,196.83
Australia   117,049.64
Germany      98,530.52
Canada       87,930.35
Denmark      80,217.33
France       54,394.89
Hungary      51,041.00
Austria      43,623.38
Italy        36,427.94
Belgium      35,664.00
Brazil       25,347.42
Name: Associate degree (A.A., A.S., etc.), dtype: float64

In [23]:
# In which of them does someone with a doctoral degree earn the most?
# In which of these countries does someone with an associate degree earn the most? 
(
    oecd_df
    .join(so_df
          .set_index('Country'))
    .pivot_table(index='Country',
                 columns='EdLevel', 
                 values='ConvertedCompYearly')['Other doctoral degree (Ph.D., Ed.D., etc.)']
    .sort_values(ascending=False)
)

Country
Japan       157,239.40
Australia   150,234.96
France      140,402.86
Israel      131,812.62
Germany     108,718.46
Canada      102,989.35
Denmark     102,785.19
Italy        93,490.78
Belgium      80,832.44
Austria      74,783.17
Finland      61,508.25
Hungary      52,833.60
Brazil       43,123.21
Name: Other doctoral degree (Ph.D., Ed.D., etc.), dtype: float64

In [24]:
# Remove rows from `so_df` in which `LanguageHaveWorkedWith` is `NaN`.
so_df = (so_df
         .dropna(subset=['LanguageHaveWorkedWith'])
        )

In [25]:
# Remove rows from `so_df` in which Python isn't included 
# as a commonly used language (`LanguageHaveWorkedWith`).
so_df = (
    so_df
    .loc[so_df['LanguageHaveWorkedWith']
    .str.contains('Python')]
)

In [26]:
# Remove rows from `so_df` in which `YearsCode` is `NaN`.
so_df = (so_df
         .dropna(subset=['YearsCode'])
        )

In [28]:
so_df.loc[so_df['YearsCode'] == 'Less than 1 year', 'YearsCode'] = 0
so_df.loc[so_df['YearsCode'] == 'More than 50 years', 'YearsCode'] = 51

In [29]:
so_df['YearsCode'] = so_df['YearsCode'].astype(int)


In [30]:
# Create a new column in `so_df`, called `experience`, which will categorize the values in `YearsCode`

so_df['experience'] = pd.cut(so_df['YearsCode'],
       bins=[-1, 1, 2, 5, 10, 100],
      labels=['Less than 1 year', 
      '1-2 years', 
      '3-5 years',
      '6-10 years',
      '11+ years'])

In [31]:

# According to the Python survey, what proportion of Python developers have each level of experience?
so_df['experience'].value_counts(normalize=True)

experience
11+ years          0.37
6-10 years         0.32
3-5 years          0.22
1-2 years          0.05
Less than 1 year   0.04
Name: proportion, dtype: float64